In [ ]:
out=pd.read_csv("/kaggle/input/merge-twitter/truth_spanish.txt",header=None,names=["label"])

In [ ]:
out.head()

In [ ]:
out['new_label']=out.label.apply(lambda x:x[-1:])
out.head()

In [ ]:
out['profile_id']=out.label.apply(lambda x:x[:-4])
out.head()

In [ ]:
print(out['new_label'].describe())

In [ ]:
import pandas as pd
import xml.etree.ElementTree as et
def parse_XML(df_cols):
    #path = '/kaggle/input/merge-twitter/'
    path = '/kaggle/input/twiter-profile-spanish/'
    rows = []
    ids = []
    new = []
    for filename in os.listdir(path):
        if not filename.endswith('.xml'): continue
        xml_file = os.path.join(path, filename)
        xtree = et.parse(xml_file)
        xroot = xtree.getroot() 
        xchild=xroot.getchildren()
        for node in xchild:
            res = []
            qs=(node.findall("document"))
            for q in qs:
                res.append(q.text)
            rows.append({i: res[i] for i, _ in enumerate(df_cols)})   
            for index, row in out.iterrows():
                if(row['profile_id']==filename[:-4]):
                    new.append({"profile_id":row['profile_id'],"truth_val":row['new_label']})
                    out_df2=pd.DataFrame(new)
                
            ids.append({id:filename[:-4]})
            out_df = pd.DataFrame(rows)
        out_df['id']=ids
    return out_df,out_df2   

In [ ]:
df,df2=parse_XML(["document","document","document","document","document","document","document","document","document","document",
"document","document","document","document","document","document","document","document","document","document",
"document","document","document","document","document","document","document","document","document","document",
"document","document","document","document","document","document","document","document","document","document",
"document","document","document","document","document","document","document","document","document","document",
"document","document","document","document","document","document","document","document","document","document",
"document","document","document","document","document","document","document","document","document","document",
"document","document","document","document","document","document","document","document","document","document",
"document","document","document","document","document","document","document","document","document","document",
"document","document","document","document","document","document","document","document","document","document"])

In [ ]:
pd.set_option('display.max_colwidth',13000)

In [ ]:
result=pd.concat([df,df2],axis=1)

In [ ]:
import re
def pre_process_data(documents,max_len=200):
    for data in documents:
        review = re.sub('[^a-zA-Z]', ' ', data)
        url = re.compile(r'https?://\S+|www\.\S+')
        review = url.sub(r'',review)
        html=re.compile(r'<.*?>')
        review = html.sub(r'',review)
        emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
        text = emoji_pattern.sub(r'',review)
        text = text[:max_len-2]

    return text    

In [ ]:
'''
for col in result:
    if col == 'id'or col=='profile_id' or col=='truth_val':
        continue
    for i, row_value in result[col].iteritems():
        result[col][i] = pre_process_data(row_value)
'''   

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
%%time
module_url = 'https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3'
embed = hub.KerasLayer(module_url, name='USE_embedding')

In [ ]:
new_result=np.zeros((300,103,512))

In [ ]:
for col in result:
    if col == 'id'or col=='profile_id' or col=='truth_val':
        continue
    for i, row_value in result[col].iteritems():
        result[col][i] = embed([row_value])

In [ ]:
result

In [ ]:
result.shape

In [ ]:
new_result=new_result[:,0:100,:]

In [ ]:
output_new=result.loc[:,'truth_val']

In [ ]:
output_new=np.asarray(output_new)

In [ ]:
result_new=result.iloc[:,0:100]#iloc takes integer for column slicing ,loc takes index of columns for slicing

In [ ]:
result_new=np.asarray(result_new)

In [ ]:
for i in range(0,300):
    for j in range(0,100):
        for k in range(512):
            new_result[i,j,k]=result_new[i][j][0][k]

In [ ]:
result_new=result_new.reshape(300,100,1)

In [ ]:
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import LSTM,Bidirectional,TimeDistributed

In [ ]:
from keras.layers import Layer
import keras.backend as K

In [ ]:
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [ ]:
model = Sequential()
#model.add(Input(shape=(100, 1),dtype=object))
model.add(LSTM(128,return_sequences=True,input_shape=(100,512),dropout=0.3,recurrent_dropout=0.2))
#model.add(TimeDistributed(Dense(128,activation='relu')))
#model.add(Bidirectional(LSTM(128,return_sequences=True)))
model.add(attention())
model.add(Dense(1,activation='sigmoid',trainable=True))
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True)

In [ ]:
history = model.fit(new_result, output_new, epochs=80,validation_split=0.1,batch_size=16,shuffle=True,callbacks=[checkpoint])

In [ ]:
model.save_weights("/kaggle/working/model_spanish.h5")
